In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time as time

import torch
from astropy.io import fits
from astropy.utils.data import download_file

from mpol import (
    coordinates,
    crossval,
    datasets,
    gridding,
    images,
    fourier,
    losses,
    precomposed,
)

from training_func import train

ModuleNotFoundError: No module named 'torch'

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# input parameters
###########################################################################################################################################
fname = '../data/visibilities/mock_visibilities_model_star_delta.npz' # path to the .npz file containing the observed visibilities
cell_size = 0.03 # arcseconds
npix = 128 # number of pixels per image axis
learning_rate = 0.3 # learning rate for the optimizer
# n_iter = 25 # number of iterations for the optimizer

start_from_dirty_image = False # If True, the initial BaseCube image is set to the dirty image, else to the default flat image.
###########################################################################################################################################

In [ ]:
# this is a multi-channel dataset... for demonstration purposes we'll use
# only the central, single channel
d = np.load(fname)
uu = d["uu"]
vv = d["vv"]
weight = d["weight"]
data = d["data"]
data_re = np.real(data)
data_im = np.imag(data)

# define the image dimensions, making sure they are big enough to fit all
# of the expected emission
coords = coordinates.GridCoords(cell_size=cell_size, npix=npix)
averager = gridding.DataAverager(
    coords=coords, uu=uu, vv=vv, weight=weight, data_re=data_re, data_im=data_im
)

# export to PyTorch dataset
dset = averager.to_pytorch_dataset()

dset

In [ ]:
imager = gridding.DirtyImager(
    coords=coords, uu=uu, vv=vv, weight=weight, data_re=data_re, data_im=data_im
)

# Show the dirty image
img, beam = imager.get_dirty_image(weighting="briggs", robust=0.0)
kw = {"origin": "lower", "extent": imager.coords.img_ext}
fig, ax = plt.subplots(ncols=1)
ax.imshow(np.squeeze(img), **kw)
ax.set_title("image")
ax.set_xlabel(r"$\Delta \alpha \cos \delta$ [${}^{\prime\prime}$]")
ax.set_ylabel(r"$\Delta \delta$ [${}^{\prime\prime}$]")

In [ ]:
# Show the dirty beam
img, beam = imager.get_dirty_image(weighting="briggs", robust=0.0)
kw = {"origin": "lower", "extent": imager.coords.img_ext}
fig, ax = plt.subplots(ncols=1)
ax.imshow(np.squeeze(beam), **kw)
ax.set_title("beam")
ax.set_xlabel(r"$\Delta \alpha \cos \delta$ [${}^{\prime\prime}$]")
ax.set_ylabel(r"$\Delta \delta$ [${}^{\prime\prime}$]")

In [ ]:
fig, ax = plt.subplots(nrows=1)
ax.scatter(uu, vv, s=1.5, rasterized=True, linewidths=0.0, c="k")
ax.scatter(
    -uu, -vv, s=1.5, rasterized=True, linewidths=0.0, c="k"
)  # and Hermitian conjugates
ax.set_xlabel(r"$u$ [k$\lambda$]")
ax.set_ylabel(r"$v$ [k$\lambda$]")
ax.set_title("original dataset")
ax.invert_xaxis()

In [ ]:
# create a radial and azimuthal partition
dartboard = datasets.Dartboard(coords=coords)

# create cross validator using this "dartboard"
k = 5
cv = crossval.DartboardSplitGridded(dset, k, dartboard=dartboard, seed=42)

# ``cv`` is a Python iterator, it will return a ``(train, test)`` pair of ``GriddedDataset``s for each iteration.
# Because we'll want to revisit the individual datasets
# several times in this tutorial, we're storeing them into a list
k_fold_datasets = [(train, test) for (train, test) in cv]

# k_fold_datasets[0][0].vis_gridded.shape

k_fold_datasets[0][0].ground_mask[0]

In [ ]:
fig, ax = plt.subplots(nrows=k, ncols=2, figsize=(4, 10))

for i, (train_subset, test_subset) in enumerate(k_fold_datasets):

    # train_subset and test_subset are `GriddedDataset`s

    train_mask = train_subset.ground_mask[0]
    test_mask = test_subset.ground_mask[0]

    ax[i, 0].imshow(
        train_mask.detach().numpy(),
        interpolation="none",
        origin="lower",
        extent=coords.vis_ext,
        cmap="GnBu",
    )

    ax[i, 1].imshow(
        test_mask.detach().numpy(), origin="lower", extent=coords.vis_ext, cmap="GnBu"
    )

    ax[i, 0].set_ylabel("k-fold {:}".format(i))

ax[0, 0].set_title("train mask")
ax[0, 1].set_title("test mask")

for a in ax.flatten():
    a.xaxis.set_ticklabels([])
    a.yaxis.set_ticklabels([])

fig.subplots_adjust(left=0.15, hspace=0.0, wspace=0.2)

In [ ]:
def test(model, dset):
    model.train(False)
    # evaluate test score
    vis = model()
    loss = losses.nll_gridded(vis, dset)
    return loss.item()

In [ ]:
def cross_validate(config):
    """
    config is a dictionary that should contain ``lr``, ``lambda_sparsity``, ``lambda_TV``, ``epochs``
    """
    test_scores = []

    for k_fold, (train_dset, test_dset) in enumerate(k_fold_datasets):

        # create a new model and optimizer for this k_fold
        rml = precomposed.SimpleNet(coords=coords, nchan=train_dset.nchan)
        optimizer = torch.optim.Adam(rml.parameters(), lr=config["lr"])

        # train for a while
        # train(rml, train_dset, config, optimizer)
        loss_tracker = train(rml=rml, dset=train_dset, hyperparams_config=config, optimizer=optimizer)
        # evaluate the test metric
        test_scores.append(test(rml, test_dset))

    # aggregate all test scores and sum to evaluate cross val metric
    test_score = np.sum(np.array(test_scores))

    return test_score

In [ ]:
def train_and_image(pars):
    rml = precomposed.SimpleNet(coords=coords, nchan=dset.nchan)
    optimizer = torch.optim.Adam(rml.parameters(), lr=pars["lr"])
    writer = SummaryWriter()
    # train(rml, dset, pars, optimizer, writer=writer)
    loss_tracker = train(rml=rml, dset=dset, hyperparams_config=pars, optimizer=optimizer, writer=writer)
    writer.close()

    img_ext = rml.coords.img_ext
    fig, ax = plt.subplots()
    ax.imshow(
        np.squeeze(rml.icube.sky_cube.detach().numpy()), origin="lower", extent=img_ext
    )
    return fig, ax

In [ ]:
pars = {'lr': 0.5,
        'lambda_sparsity': 1.0e-4,
        'lambda_TV': 1.0e-4,
        'entropy': 1.0e-4,
        'prior_intensity': 1.0e-7,
        'TSV': 1.0e-6,
        "epochs": 600}

print("Cross validation score:", cross_validate(pars))
train_and_image(pars)

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
# Define missing variables
lambda_sparsity_space = np.logspace(-6, -2, 4)
lambda_TV_space = np.logspace(-8, -4, 3)
# lambda_TV_space = np.array([1.0e-6])
entropy_space = np.logspace(-7, -2, 4)
TSV_space = np.logspace(-6, -2, 4)
torch.set_num_threads(10)


# Fix code block
cv_dataframe = pd.DataFrame(columns=['lambda_sparsity', 'lambda_TV', 'entropy', 'TSV', 'cross_val_score'])
parameter_total = len(lambda_sparsity_space)*len(lambda_TV_space)*len(entropy_space)*len(TSV_space)
start_time = time.time()
for sparsity_index, lambda_sparsity in enumerate(lambda_sparsity_space):
    for TV_index, lambda_TV in enumerate(lambda_TV_space):
        for entropy_index, entropy in enumerate(entropy_space):
            for TSV_index, TSV in enumerate(TSV_space):
                parameter_count=sparsity_index*len(lambda_TV_space)*len(entropy_space)*len(TSV_space)+TV_index*len(entropy_space)*len(TSV_space)+entropy_index*len(TSV_space)+TSV_index+1
                # print(f"[{parameter_count}/{parameter_total}]")
                run_time_start = time.time()
                pars = {'lr': 0.5,
                        'lambda_sparsity': lambda_sparsity,
                        'lambda_TV': lambda_TV,
                        'entropy': entropy,
                        'prior_intensity': 1.0e-7,
                        'TSV': TSV,
                        "epochs": 600}
                cv_score = cross_validate(pars)
                print("[{}/{}] --> CV: {:.2f}, log_TSV:{:.1e}, log_entropy:{:.1e}, log_TV:{:.1e}, log_sparsity:{:.1e}".format(parameter_count, parameter_total, cv_score, np.log10(TSV), np.log10(entropy), np.log10(lambda_TV), np.log10(lambda_sparsity)))
                cv_dataframe = pd.concat([cv_dataframe, pd.DataFrame({'lambda_sparsity': lambda_sparsity, 'lambda_TV': lambda_TV, 'entropy': entropy, 'TSV': TSV, 'cross_val_score': cv_score}, index=[0])], ignore_index=True)
                # train_and_image(pars)
                run_time_end = time.time()
                predicted_time = (time.time()-start_time)*(parameter_total-parameter_count)/(parameter_count)
                # print("runtime: {:.1f} seconds \t remaining time: {:.1f} minutes\n".format(run_time_end-run_time_start, predicted_time/60))

end_time = time.time()
print("Time taken for cross validation:{:.2f} minutes".format((end_time-start_time)/60))
cv_dataframe.to_csv('cv_dataframe.csv', index=True)

In [ ]:
cv_dataframe.to_csv('cv_dataframe.csv', index=True)

In [ ]:
cv_dataframe

In [ ]:
cv_dataframe.sort_values(by='cross_val_score', ascending=True)

In [ ]:
idx = cv_dataframe['cross_val_score'].idxmin()
print("lowest cross validation score:\n", cv_dataframe.loc[idx])

In [ ]:
pars = {'lr': 0.5,
        'lambda_sparsity': cv_dataframe['lambda_sparsity'][idx],
        'lambda_TV': cv_dataframe['lambda_TV'][idx],
        'entropy': cv_dataframe['entropy'][idx],
        'prior_intensity': 1.0e-7,
        'TSV': cv_dataframe['TSV'][idx],
        "epochs": 600}

train_and_image(pars)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2e' % x)
interest_cols = ['lambda_sparsity', 'entropy', 'TSV', 'cross_val_score']
df= cv_dataframe
df_interest = df[interest_cols]
interest_cols = ['lambda_sparsity', 'entropy', 'TSV']
print(df_interest.sort_values(by='cross_val_score', ascending=True))

# plot a contour plot for each two parameter combination and the cross validation score. 
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
for i, col in enumerate(interest_cols):
    X = df_interest[interest_cols[i-1]]
    Y = df_interest[interest_cols[i]]
    Z = df_interest['cross_val_score']
    ax[i].tricontourf(X, Y, Z, levels=14, cmap='RdYlBu')
    ax[i].set_xscale('log')
    ax[i].set_yscale('log')
    ax[i].set_xlabel(interest_cols[i-1])
    ax[i].set_ylabel(interest_cols[i])
    ax[i].set_title('Cross validation score')
    fig.colorbar(ax[i].tricontourf(X, Y, Z, levels=14), ax=ax[i], orientation='horizontal')
plt.show()